In [2]:
# 0. Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1. Import Libraries
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.metrics import classification_report, roc_auc_score
from scipy.stats.mstats import winsorize
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Dropout, Lambda
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Set Seed untuk Konsistensi
SEED = 44
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# 3. Load Dataset
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Skripsi/Dataset/diabetes_012_health_indicators_BRFSS2015.csv')
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

# 4. Preprocessing
X['BMI'] = winsorize(X['BMI'], limits=[0.005, 0.005])
X['MentHlth'] = np.where(X['MentHlth'] > 30, 30, X['MentHlth'])
X['PhysHlth'] = np.where(X['PhysHlth'] > 30, 30, X['PhysHlth'])

robust_features = ['BMI', 'MentHlth', 'PhysHlth']
minmax_features = ['Age', 'Education', 'Income', 'GenHlth']

scaler_robust = RobustScaler()
scaler_minmax = MinMaxScaler()
X[robust_features] = scaler_robust.fit_transform(X[robust_features])
X[minmax_features] = scaler_minmax.fit_transform(X[minmax_features])

# 5. One-hot Encoding untuk Target
y_encoded = to_categorical(y)

# 6. Split Data (Train/Test -> Train/Val)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=SEED, stratify=y
)
X_train_main, X_val, y_train_main, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train.argmax(axis=1)
)

# 7. Build DNN + Self-Attention Layer (Moved before Output Layer)
input_layer = Input(shape=(X.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# Expand dimensi untuk Self-Attention
x_expanded = Lambda(lambda t: tf.expand_dims(t, axis=1))(x)

# Self-Attention sebelum output
attn_output = MultiHeadAttention(num_heads=4, key_dim=4)(x_expanded, x_expanded)
attn_output = LayerNormalization()(attn_output + x_expanded)

# Kembalikan ke bentuk 2D
attn_output_squeezed = Lambda(lambda t: tf.squeeze(t, axis=1))(attn_output)

# Output layer
output_layer = Dense(3, activation='softmax')(attn_output_squeezed)

# Final Model
model_sa = Model(inputs=input_layer, outputs=output_layer)
model_sa.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 8. Train Model (Updated Epochs: 100)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_sa = model_sa.fit(
    X_train_main, y_train_main,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=50,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluate
y_pred_prob_sa = model_sa.predict(X_test)
y_pred_class_sa = np.argmax(y_pred_prob_sa, axis=1)
y_true_class_sa = np.argmax(y_test, axis=1)

print("\n=== Classification Report (Macro) DNN + SA ===")
print(classification_report(y_true_class_sa, y_pred_class_sa, digits=4))
roc_auc_sa = roc_auc_score(y_test, y_pred_prob_sa, average='macro', multi_class='ovr')
print(f"\nMacro-average ROC AUC: {roc_auc_sa:.4f}")

# 12. Simpan Model dalam Format .keras
save_dir = '/content/drive/MyDrive/Projects/DNN untuk Prediksi status diabetes/Model'
os.makedirs(save_dir, exist_ok=True)

model_path = os.path.join(save_dir, 'dnn+self-attention.keras')
model_sa.save(model_path)

print(f"Model berhasil disimpan di: {model_path}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step - accuracy: 0.8365 - loss: 0.4330 - val_accuracy: 0.8489 - val_loss: 0.4013
Epoch 2/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8478 - loss: 0.3975 - val_accuracy: 0.8490 - val_loss: 0.3999
Epoch 3/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8486 - loss: 0.3957 - val_accuracy: 0.8494 - val_loss: 0.3999
Epoch 4/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.8489 - loss: 0.3947 - val_accuracy: 0.8489 - val_loss: 0.3992
Epoch 5/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.8488 - loss: 0.3940 - val_accuracy: 0.8489 - val_loss: 0.3990
Epoch 6/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.8495 - loss: 0.3934 - val_accuracy: 0.8488 - val_loss: 0.3985
Epoch 7/100
3248/3248 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.8496 - lo

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
